In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm_notebook as tqdm
import datetime
from sklearn import metrics
import gc
import os

In [ ]:
building = pd.read_csv('/content/drive/MyDrive/building_metadata.csv')
weather = pd.read_csv('/content/drive/MyDrive/weather_train.csv')
train = pd.read_csv('/content/drive/MyDrive/train.csv')
# weather_test = pd.read_csv('/content/drive/MyDrive/weather_test.csv')
# test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
import keras.backend as K

In [ ]:
K.clear_session()

In [ ]:
building.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [ ]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [ ]:
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


### Nombre de valeurs manquantes

In [ ]:
print('Nombre des valeurs manquantes dans train:' ,np.sum(train.isnull().values))
# print('Nombre des valeurs manquantes dans test:' ,np.sum(test.isnull().values))
print('Nombre des valeurs manquantes dans weather',np.sum(weather.isnull().values))
# print('Nombre des valeurs manquantes dans weather_test',np.sum(weather_test.isnull().values))
print('Nombre des valeurs manquantes dans building:',np.sum(building.isnull().values))

Nombre des valeurs manquantes dans train: 0
Nombre des valeurs manquantes dans weather 136820
Nombre des valeurs manquantes dans building: 1868


### Fonction pour réduire la mémoire

In [ ]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
building = reduce_mem_usage(building)
weather = reduce_mem_usage(weather)
train = reduce_mem_usage(train)
# weather_test = reduce_mem_usage(weather_test)
# test = reduce_mem_usage(test)

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 65.4%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 4.51 MB
Decreased by 53.0%
Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%


## On remplit les données  de weather

In [ ]:
!pip install meteocalc

In [ ]:
from meteocalc import feels_like, Temp

In [ ]:
def fill_weather_dataset(weather_df):
    
    # Add Day,Week & Month features This dataset consits of hourly weather information. 
    # So we are going to fill missing values based on below new date features.
    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    # adding day ,moth and week as features
    weather_df["day"] = weather_df["datetime"].dt.day 
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])
     
    # fill missing air temperature with mean temperature of day of the month. 
    # Each month comes in a season and temperature varies lots in a season. So filling with yearly mean value is not a good idea.
    
    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)


    #Data is missing for most of days and even many consecutive days. So, first, calculate mean cloud_coverage of day of the month 
    # then fill rest missing values with last valid observation.(fillna with the method='ffill')
    
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"]) # imputing with daily means per site id
    weather_df.update(cloud_coverage_filler,overwrite=False)

    #fillna with the method='ffill' option. 'ffill' stands for 'forward fill' and will propagate last valid observation forward
    
    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)  

    
    # Data is missing for most of days and even many consecutive days. So, first, calculate mean sea_level of day of the month 
    # then fill rest missing values with last valid observation.(fillna with the method='ffill')
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])
    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)


    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])
    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
    # weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    return weather_df

### meteorological_features

In [ ]:
def calculate_rh(df):
  df['relative_humidity'] = 100 * (np.exp((17.625 * df['dew_temperature']) / (243.04 + df['dew_temperature'])) / np.exp((17.625 * df['air_temperature'])/(243.04 + df['air_temperature'])))

def calculate_fl(df):
  flike_final = []
  flike = []
  # calculate Feels Like temperature
  for i in range(len(df)):
      at = df['air_temperature'][i]
      rh = df['relative_humidity'][i]
      ws = df['wind_speed'][i]
      flike.append(feels_like(Temp(at, unit = 'C'), rh, ws))
  for i in range(len(flike)):
      flike_final.append(flike[i].f)
  df['feels_like'] = flike_final
  del flike_final, flike, at, rh, ws

def get_meteorological_features(data):
  calculate_rh(data)
  calculate_fl(data)
  return data


  


### Feature engineering

In [ ]:

def features_engineering(df):
    
  # Sort by timestamp
  df.sort_values("timestamp")
  df.reset_index(drop=True)
  
  # Add more features
  df["timestamp_2"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
  df["hour"] = df["timestamp_2"].dt.hour
  df["dayofweek"] = df["timestamp_2"].dt.weekday

  df['group'] = df['timestamp_2'].dt.month
  df['group'].replace((1, 2, 3, 4), 1, inplace = True)
  df['group'].replace((5, 6, 7, 8), 2, inplace = True)
  df['group'].replace((9, 10, 11, 12), 3, inplace = True)
  df = df.drop('timestamp_2', 1)
  # df["timestamp"] = pd.to_datetime(df["timestamp"], utc = True)
  return df

In [ ]:
weather_df = fill_weather_dataset(weather)
weather_df = get_meteorological_features(weather_df)

# weather_test_df = fill_weather_dataset(weather_test)
# weather_test_df = get_meteorological_features(weather_test_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  if __name__ == '__main__':


In [ ]:
weather_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,relative_humidity,feels_like
0,0,2016-01-01 00:00:00,25.000000,6.000000,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.000000
1,0,2016-01-01 01:00:00,24.400000,4.285714,21.1,-1.000000,1020.200012,70.0,1.5,81.864394,75.919999
2,0,2016-01-01 02:00:00,22.799999,2.000000,21.1,0.000000,1020.200012,0.0,0.0,90.149092,73.039999
3,0,2016-01-01 03:00:00,21.100000,2.000000,20.6,0.000000,1020.099976,0.0,0.0,96.971205,69.980001
4,0,2016-01-01 04:00:00,20.000000,2.000000,20.0,-1.000000,1020.000000,250.0,2.6,100.000000,68.000000


In [ ]:
print('Nombre des valeurs manquantes dans train:' ,np.sum(train.isnull().values))
print('Nombre des valeurs manquantes dans weather',np.sum(weather_df.isnull().values))
# print('Nombre des valeurs manquantes dans weather_test',np.sum(weather_test_df.isnull().values))
print('Nombre des valeurs manquantes dans building:',np.sum(building.isnull().values))

Nombre des valeurs manquantes dans train: 0
Nombre des valeurs manquantes dans weather 0
Nombre des valeurs manquantes dans building: 1868


In [ ]:
weather_df  = features_engineering(weather_df)
# weather_test_df = features_engineering(weather_test_df)
train_df  = features_engineering(train)
# test_df  = features_engineering(test)

### Vu qu'on a les données avec des dates, il pourait être intéressant d'ajouter une autre variable catégorielle binaire qui donnera pour chaque date si ça correspond à une date de congés, ...

In [ ]:
location=pd.DataFrame()
location['site_id']=np.arange(0,16)


location['city']=['Orlando','Heathrow','Tempe','Washington','Berkeley','Southampton',\
                     'Washington','Ottowa','Orlando','Austin','Saltlake','Ottowa','Dublin',\
                      'Minneapolis','Philadelphia','Rochester']
location['country']=['US','UK','US','US','US','UK',\
                    'US','Montreal','US','US','US','Montreal','Ireland',\
                    'US','US','US']
weather_df= weather_df.merge(location, on='site_id', how='left')
# weather_test_df= weather_test_df.merge(location, on='site_id', how='left')

In [ ]:
from datetime import date
## https://www.geeksforgeeks.org/python-holidays-library/
## https://towardsdatascience.com/5-minute-guide-to-detecting-holidays-in-python-c270f8479387
import holidays
UK=[]
for ptr in holidays.UnitedKingdom(years=2016).keys():
    UK.append(str(ptr))
for ptr in holidays.UnitedKingdom(years=2017).keys(): #2017 year holydays in uk
    UK.append(str(ptr))
for ptr in holidays.UnitedKingdom(years=2018).keys():
    UK.append(str(ptr))
    UK.append('2019-01-01')
IR=[]
for ptr in holidays.Ireland(years=2016).keys():  #2016 year holydays in ireland
    IR.append(str(ptr))
for ptr in holidays.Ireland(years=2017).keys():
    IR.append(str(ptr))
for ptr in holidays.Ireland(years=2018).keys():
    IR.append(str(ptr))
    IR.append('2019-01-01')
US=[]
for ptr in holidays.UnitedStates(years=2016).keys(): #2016 year holydays in US
    US.append(str(ptr))
for ptr in holidays.UnitedStates(years=2017).keys():
    US.append(str(ptr))
for ptr in holidays.UnitedStates(years=2018).keys():
    US.append(str(ptr))
    US.append('2019-01-01')
CA=[]
for ptr in holidays.Canada(years=2016).keys():   #2016 year holydays in Canada
    CA.append(str(ptr))
for ptr in holidays.Canada(years=2017).keys():
    CA.append(str(ptr))
for ptr in holidays.Canada(years=2018).keys():
    CA.append(str(ptr))
    CA.append('2019-01-01')

In [ ]:

def is_holiday(df):
    df['is_holiday']=[0]*(df.shape[0])
    timestamp = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df.loc[df['country']=='US','is_holiday']=(timestamp.dt.date.astype('str').isin(US)).astype(int)
    df.loc[df['country']=='UK','is_holiday']=(timestamp.dt.date.astype('str').isin(UK)).astype(int)
    df.loc[df['country']=='Montreal','is_holiday']=(timestamp.dt.date.astype('str').isin(CA)).astype(int)
    df.loc[df['country']=='Ireland','is_holiday']=(timestamp.dt.date.astype('str').isin(IR)).astype(int)
    return df
# get the holiday
weather_df=is_holiday(weather_df)
# weather_test_df=is_holiday(weather_test_df)

In [ ]:
train.head()

,building_id,meter,timestamp,meter_reading,timestamp_2,hour,dayofweek,group
0,0,0,2016-01-01 00:00:00,0.0,2016-01-01,0,4,1
1,1,0,2016-01-01 00:00:00,0.0,2016-01-01,0,4,1
2,2,0,2016-01-01 00:00:00,0.0,2016-01-01,0,4,1
3,3,0,2016-01-01 00:00:00,0.0,2016-01-01,0,4,1
4,4,0,2016-01-01 00:00:00,0.0,2016-01-01,0,4,1


In [ ]:
weather_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,relative_humidity,feels_like,hour,dayofweek,group,city,country,is_holiday
0,0,2016-01-01 00:00:00,25.000000,6.000000,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.000000,0,4,1,Orlando,US,1
1,0,2016-01-01 01:00:00,24.400000,4.285714,21.1,-1.000000,1020.200012,70.0,1.5,81.864394,75.919999,1,4,1,Orlando,US,1
2,0,2016-01-01 02:00:00,22.799999,2.000000,21.1,0.000000,1020.200012,0.0,0.0,90.149092,73.039999,2,4,1,Orlando,US,1
3,0,2016-01-01 03:00:00,21.100000,2.000000,20.6,0.000000,1020.099976,0.0,0.0,96.971205,69.980001,3,4,1,Orlando,US,1
4,0,2016-01-01 04:00:00,20.000000,2.000000,20.0,-1.000000,1020.000000,250.0,2.6,100.000000,68.000000,4,4,1,Orlando,US,1


In [ ]:
train_df = train_df.merge(building, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])

# test_df = test_df.merge(building,left_on='building_id',right_on='building_id',how='left')
# del building
# gc.collect()



# test_df = test_df.merge(weather_test_df,how='left',on=['timestamp','site_id'])
# del weather_df
# gc.collect()


# transform target variable
train_df['meter_reading'] = np.log1p(train_df["meter_reading"])
train_df['square_feet'] =  np.log1p(train_df['square_feet'])

# test_df['square_feet'] =  np.log1p(test_df['square_feet'])
    
    # Remove Unused Columns
     # Encode Categorical Data
le = LabelEncoder()
train_df["primary_use"] = le.fit_transform(train_df["primary_use"])
# test_df["primary_use"] = le.fit_transform(test_df["primary_use"])

In [ ]:
train_df.head()

,building_id,meter,timestamp,meter_reading,hour_x,dayofweek_x,group_x,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,relative_humidity,feels_like,hour_y,dayofweek_y,group_y,city,country,is_holiday
0,0,0,2016-01-01 00:00:00,0.0,0,4,1,0,0,8.913685,2008.0,NaN,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.0,0.0,4.0,1.0,Orlando,US,1.0
1,1,0,2016-01-01 00:00:00,0.0,0,4,1,0,0,7.908755,2004.0,NaN,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.0,0.0,4.0,1.0,Orlando,US,1.0
2,2,0,2016-01-01 00:00:00,0.0,0,4,1,0,0,8.589886,1991.0,NaN,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.0,0.0,4.0,1.0,Orlando,US,1.0
3,3,0,2016-01-01 00:00:00,0.0,0,4,1,0,0,10.072639,2002.0,NaN,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.0,0.0,4.0,1.0,Orlando,US,1.0
4,4,0,2016-01-01 00:00:00,0.0,0,4,1,0,0,11.666573,1975.0,NaN,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.0,0.0,4.0,1.0,Orlando,US,1.0


### Removing outliers and floor_count feature


In [ ]:
train_df.drop('floor_count',axis=1,inplace=True)
# test_df.drop('floor_count',axis=1,inplace=True)

### Removing rows with electricity = 0

In [ ]:

idx_to_drop = list(train_df[(train_df['meter'] == 0) & (train_df['meter_reading'] == 0)].index)
train_df.drop(idx_to_drop,axis='rows',inplace=True)

### Suppression de la ligne du building_id 1099 considéré outier lors de l'analyse des données

In [ ]:
idx_to_drop = list(train_df[(train_df['building_id'] == 1099)].index)
train_df.drop(idx_to_drop,axis='rows',inplace=True)

In [ ]:
train_df.head()

,building_id,meter,timestamp,meter_reading,hour_x,dayofweek_x,group_x,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,relative_humidity,feels_like,hour_y,dayofweek_y,group_y,city,country,is_holiday
45,46,0,2016-01-01 00:00:00,3.993413,0,4,1,0,11,9.110078,2016.0,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.000000,0.0,4.0,1.0,Orlando,US,1.0
72,74,0,2016-01-01 00:00:00,3.784219,0,4,1,0,8,12.867830,1997.0,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.000000,0.0,4.0,1.0,Orlando,US,1.0
91,93,0,2016-01-01 00:00:00,3.978196,0,4,1,0,6,10.415443,1982.0,25.0,6.0,20.0,-0.173913,1019.700012,0.0,0.0,73.802512,77.000000,0.0,4.0,1.0,Orlando,US,1.0
103,105,0,2016-01-01 00:00:00,3.190624,0,4,1,1,0,10.832181,NaN,3.8,0.0,2.4,0.000000,1020.900024,240.0,3.1,90.575534,36.933764,0.0,4.0,1.0,Heathrow,UK,1.0
104,106,0,2016-01-01 00:00:00,0.318163,0,4,1,1,0,8.589514,NaN,3.8,0.0,2.4,0.000000,1020.900024,240.0,3.1,90.575534,36.933764,0.0,4.0,1.0,Heathrow,UK,1.0


In [ ]:
# train_df = train_df.drop(['timestamp','hour_y','dayofweek_y','group_y','hour_x'], axis = 1)

train_df = train_df.drop(['timestamp','country','city','hour_y','dayofweek_y','group_y','hour_x'], axis = 1)
# test_df = test_df.drop(['timestamp','country','city','hour_y','dayofweek_y','group_y','hour_x'], axis = 1)

In [ ]:
# test_df.to_csv('/content/drive/MyDrive/test_preprocessed1.csv')
train_df.to_csv('/content/drive/MyDrive/train_preprocessed1.csv')